In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import argparse
import numpy as np
import os
from qonnx.core.datatype import DataType
from driver_base import FINNExampleOverlay
from pynq.pl_server.device import Device

In [ ]:
'''
UNET 3-LEVEL ARGS
/**
 *  ap_uint<8> block			    = 1,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 6,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 6,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 2,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 16,	|	
    ap_uint<8> weight_in_simd_arg	= 8,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg        = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 2,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 6,	|	ap_uint<1> MaxPooling_en	= 1,
	ap_uint<4> OFMDim_arg		    = 3,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 16,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 16,	|	
    ap_uint<8> weight_in_simd	    = 16,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg        = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 3,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 3,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 3,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 16,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 32,	|	
    ap_uint<8> weight_in_simd	    = 16,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg        = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 4,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 3,	|	ap_uint<1> MaxPooling_en	= 1,
	ap_uint<4> OFMDim_arg		    = 1,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 32,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 32,	|	
    ap_uint<8> weight_in_simd	    = 32,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg        = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 1
 */

 /**
 *  ap_uint<8> block			    = 5,	|	ap_uint<2> nf_compute		= 2,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 1,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 1,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 32,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 64,	|	
    ap_uint<8> weight_in_simd	    = 32,
    ap_uint<8> MVAU_Tiles_arg       = 18
    ap_uint<8> UpS_Tiles_arg        = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 6,	|	ap_uint<2> nf_compute		= 2,
                                                ap_uint<2> scale_factor_arg = 3,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 1,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 3,	|	ap_uint<1> Upsampling_en	= 1,
	ap_uint<8> IFMChannel_arg	    = 64,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 64,	|	
    ap_uint<8> weight_in_simd	    = 64,
    ap_uint<8> MVAU_Tiles_arg       = 18
    ap_uint<8> UpS_Tiles_arg        = 1
	ap_uint<8> OUPChannel_arg	    = 32,	|	ap_uint<2> buf_index		= 1
 */

 /**
 *  ap_uint<8> block			    = 7,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 3,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 3,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 32,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 16,	|	
    ap_uint<8> weight_in_simd	    = 32,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg_1      = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 8,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 2,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 3,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 6,	|	ap_uint<1> Upsampling_en	= 1,
	ap_uint<8> IFMChannel_arg	    = 16,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 16,	|	
    ap_uint<8> weight_in_simd	    = 16,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg_1      = 1
	ap_uint<8> OUPChannel_arg	    = 16,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 9,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 6,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 6,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 16,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 8,	|	
    ap_uint<8> weight_in_simd	    = 16,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg_1      = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */

 /**
 *  ap_uint<8> block			    = 10,	|	ap_uint<2> nf_compute		= 1,
                                                ap_uint<2> scale_factor_arg = 0,
                                                ap_uint<2> Padding_arg      = 0,
	ap_uint<4> IFMDim_arg		    = 6,	|	ap_uint<1> MaxPooling_en	= 0,
	ap_uint<4> OFMDim_arg		    = 6,	|	ap_uint<1> Upsampling_en	= 0,
	ap_uint<8> IFMChannel_arg	    = 8,	|	
	ap_uint<8> MVAU_OFMChannel_arg	= 8,	|	
    ap_uint<8> weight_in_simd	    = 8,
    ap_uint<8> MVAU_Tiles_arg       = 9
    ap_uint<8> UpS_Tiles_arg_1      = 0
	ap_uint<8> OUPChannel_arg	    = 0,	|	ap_uint<2> buf_index		= 0
 */
'''

In [ ]:
'''
# test_top
(0x10, block_r)
(0x18, IFMDim_arg)
(0x20, OFMDim_arg)
(0x28, IFMChannel_arg)
(0x30, MVAU_OFMChannel_arg)
(0x38, weight_in_simd)
(0x40, MVAU_Tiles_arg)
(0x48, UpS_Tiles_arg)
(0x50, OUPChannel_arg)
(0x58, nf_compute)
(0x60, UpS_nf_compute)
(0x68, MaxPooling_en)
(0x70, Upsampling_en)
(0x78, buf_index)


# idma_weight_1
(0x10, iwbuf_1.device_address)
(0x00, 1)

# idma_weight_2
(0x10, iwbuf_2.device_address)
(0x00, 1)

# odma
(0x10, obuf_packed_device.device_address)
(0x00, 1)

# idma
(0x10, ibuf_packed_device.device_address)
(0x00, 1)
'''

In [ ]:
def block(self, ind=0):
    return io_shape_dict["block"][ind]

In [ ]:
import os
import numpy as np

def load_weights():
    w_filenames = []
    runtime_weight_dir = "MVAU_weights_npy/"
    if not os.path.isdir(runtime_weight_dir):
        return
    for dirpath, dirnames, filenames in os.walk(runtime_weight_dir):
        w_filenames.extend(filenames)

    w_filenames = sorted(w_filenames, key=lambda x: int(x.split('_')[1].split('.')[0]))

    weight_tensor_dconv = []
    weight_tensor_ups = []
    
    for w_filename in w_filenames:
        if w_filename.endswith(".npy"):
            index = int(w_filename.split('_')[1].split('.')[0])
            #print(index)
            weight_tensor = np.load(runtime_weight_dir + "/" + w_filename)
            if index in [6, 9]:
                weight_tensor_ups.append(weight_tensor)
                print("%s : index %d, weight_tensor_shape %s" % (w_filename, index, weight_tensor.shape))
            else:
                weight_tensor_dconv.append(weight_tensor)
                #print("%s : index %d, weight_tensor_shape %s" % (w_filename, index, weight_tensor.shape))
        else:
            continue

        
    print(f"Collected filenames: {w_filenames}")
   

load_weights()

In [ ]:
# dictionary describing the I/O of the FINN-generated accelerator
fmdim = 6
io_shape_dict = {
    # FINN DataType for input and output tensors
    "idt" : [DataType['UINT4']],
    "odt" : [DataType['INT32']],
    # shapes for input and output tensors (NHWC layout)
    "ishape_normal" : [(1, fmdim, fmdim, 2)],
    "oshape_normal" : [(1, fmdim, fmdim, 1)],
    # folded / packed shapes below depend on idt/odt and input/output
    # PE/SIMD parallelization settings -- these are calculated by the
    # FINN compiler.
    "ishape_folded" : [(1, fmdim, fmdim, 1, 2)],
    "oshape_folded" : [(1, fmdim, fmdim, 1, 1)],
    "ishape_packed" : [(1, fmdim, fmdim, 1, 1)],
    "oshape_packed" : [(1, fmdim, fmdim, 1, 4)], # the last dimension should be the numBytes(8 bits)
    "input_dma_name" : ['idma_0'],
    "weight_dma_name" : ['idma_weight_1_0', 'idma_weight_2_0'],
    "accel_name" : ['top_0'],
    "output_dma_name" : ['odma_0'],
    # "number_of_external_weights": 0,
    "num_inputs" : 1,
    "num_outputs" : 1,
    # args for accel
    # (block_r, True/False) indicates if there is the second weight load operation in that block, block_r is the block number
    "block" : [(i, i in [6, 8]) for i in range(1, 11)],
    "IFMDim_arg": [6, 6, 3, 3, 1, 1, 3, 3, 6, 6],
    "OFMDim_arg": [6, 3, 3, 1, 1, 3, 3, 6, 6, 6],
    "IFMChannel_arg": [2, 16, 16, 32, 32, 64, 32, 16, 16, 8],
    "MVAU_OFMChannel_arg": [16, 16, 32, 32, 64, 64, 16, 16, 8, 8],
    "weight_in_simd_arg": [8, 16, 16, 32, 32, 64, 32, 16, 16, 8],
    "MVAU_Tiles_arg": [9, 9, 9, 9, 18, 18, 9, 9, 9, 9],
    "UpS_Tiles_arg": [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    "OUPChannel_arg": [0, 0, 0, 0, 0, 32, 0, 16, 0, 0],
    "nf_compute": [1, 1, 1, 1, 2, 2, 1, 1, 1, 1],
    "scale_factor_arg": [0, 0, 0, 0, 0, 3, 0, 2, 0, 0],
    "Padding_arg": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "MaxPooling_en": [0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
    "Upsampling_en": [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    "buf_index": [0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
}

exec_mode = "throughput_test"
platform = "zynq-iodma"
batch_size = 1
bitfile = "unet3L.bit"
runtime_weight_dir = "MVAU_weights_npy/"
devID = 0
device = Device.devices[devID]

# instantiate FINN accelerator driver and pass batchsize and bitfile
accel = FINNExampleOverlay(
    bitfile_name = bitfile, platform = platform,
    io_shape_dict = io_shape_dict, batch_size = batch_size,
    runtime_weight_dir = runtime_weight_dir, device=device
)


In [ ]:
accel.ip_dict

In [ ]:
from pynq import GPIO

# Define a function to read specific bit ranges from GPIO
def read_gpio_ranges():
    # Read bits 11:0
    gpio_pins_0_11 = [GPIO(GPIO.get_gpio_pin(i), 'in') for i in range(12)]
    value_0_11 = 0
    for i in range(12):
        bit_value = gpio_pins_0_11[i].read()
        value_0_11 |= (bit_value << i)

    # Read bits 23:12
    gpio_pins_12_23 = [GPIO(GPIO.get_gpio_pin(i), 'in') for i in range(12, 24)]
    value_12_23 = 0
    for i in range(12):
        bit_value = gpio_pins_12_23[i].read()
        value_12_23 |= (bit_value << i)

    # Read bits 29:24
    gpio_pins_24_29 = [GPIO(GPIO.get_gpio_pin(i), 'in') for i in range(24, 30)]
    value_24_29 = 0
    for i in range(6):
        bit_value = gpio_pins_24_29[i].read()
        value_24_29 |= (bit_value << i)

    # Read bits 35:30
    gpio_pins_30_35 = [GPIO(GPIO.get_gpio_pin(i), 'in') for i in range(30, 36)]
    value_30_35 = 0
    for i in range(6):
        bit_value = gpio_pins_30_35[i].read()
        value_30_35 |= (bit_value << i)

    # Return all the values
    return value_0_11, value_12_23, value_24_29, value_30_35

In [ ]:
# for the remote execution the data from the input npy file has to be loaded,
# packed and copied to the PYNQ buffer
if exec_mode == "execute":
    # load desired input .npy file(s)
    ibuf_normal = []
    for ifn in inputfile:
        ibuf_normal.append(np.load(ifn))
    obuf_normal = accel.execute(ibuf_normal) # this execute function should run all the blocks(a complete unet), 
                                             # meaning that the accel is actually running multiple times in the execute function
    if not isinstance(obuf_normal, list):
        obuf_normal = [obuf_normal]
    for o, obuf in enumerate(obuf_normal):
        np.save(outputfile[o], obuf)
else:
    raise Exception("Exec mode has to be set to execute")

input_time_accu =  0.04744529724121094
lw_time_accu =  0.4913806915283203
accel_time_accu =  0.3986358642578125
output_time_accu =  0.03838539123535156
total_time =  0.9758472442626953
input_time_accu =  0.045299530029296875
lw_time_accu =  0.43392181396484375
accel_time_accu =  0.396728515625
output_time_accu =  0.03743171691894531
total_time =  0.9133815765380859
input_time_accu =  0.04410743713378906
lw_time_accu =  0.43511390686035156
accel_time_accu =  0.39887428283691406
output_time_accu =  0.03838539123535156
total_time =  0.9164810180664062
input_time_accu =  0.04410743713378906
lw_time_accu =  0.4355907440185547
accel_time_accu =  0.3991127014160156
output_time_accu =  0.03719329833984375
total_time =  0.9160041809082031
input_time_accu =  0.04506111145019531
lw_time_accu =  0.4372596740722656
accel_time_accu =  0.40078163146972656
output_time_accu =  0.03743171691894531
total_time =  0.9205341339111328
input_time_accu =  0.04792213439941406
lw_time_accu =  0.4367828369140625


input_time_accu =  0.04696846008300781
lw_time_accu =  0.4374980926513672
accel_time_accu =  0.3974437713623047
output_time_accu =  0.03743171691894531
total_time =  0.919342041015625
input_time_accu =  0.04410743713378906
lw_time_accu =  0.4360675811767578
accel_time_accu =  0.3993511199951172
output_time_accu =  0.03790855407714844
total_time =  0.9174346923828125
input_time_accu =  0.04458427429199219
lw_time_accu =  0.43511390686035156
accel_time_accu =  0.3993511199951172
output_time_accu =  0.03814697265625
total_time =  0.9171962738037109
input_time_accu =  0.044345855712890625
lw_time_accu =  0.4363059997558594
accel_time_accu =  0.39768218994140625
output_time_accu =  0.037670135498046875
total_time =  0.9160041809082031
input_time_accu =  0.047206878662109375
lw_time_accu =  0.43773651123046875
accel_time_accu =  0.39887428283691406
output_time_accu =  0.03838539123535156
total_time =  0.9222030639648438
input_time_accu =  0.04839897155761719
lw_time_accu =  0.438451766967773

input_time_accu =  0.046253204345703125
lw_time_accu =  0.43511390686035156
accel_time_accu =  0.39696693420410156
output_time_accu =  0.03719329833984375
total_time =  0.91552734375
input_time_accu =  0.044345855712890625
lw_time_accu =  0.5309581756591797
accel_time_accu =  0.4093647003173828
output_time_accu =  0.03886222839355469
total_time =  1.0235309600830078
input_time_accu =  0.04506111145019531
lw_time_accu =  0.4367828369140625
accel_time_accu =  0.3986358642578125
output_time_accu =  0.037670135498046875
total_time =  0.9181499481201172
input_time_accu =  0.044345855712890625
lw_time_accu =  0.45418739318847656
accel_time_accu =  0.40411949157714844
output_time_accu =  0.03790855407714844
total_time =  0.9405612945556641
input_time_accu =  0.04506111145019531
lw_time_accu =  0.4372596740722656
accel_time_accu =  0.39577484130859375
output_time_accu =  0.03743171691894531
total_time =  0.91552734375
input_time_accu =  0.04649162292480469
lw_time_accu =  0.45228004455566406
a

input_time_accu =  0.04649162292480469
lw_time_accu =  0.43702125549316406
accel_time_accu =  0.3962516784667969
output_time_accu =  0.03719329833984375
total_time =  0.9169578552246094
input_time_accu =  0.04267692565917969
lw_time_accu =  0.43487548828125
accel_time_accu =  0.39768218994140625
output_time_accu =  0.037670135498046875
total_time =  0.9129047393798828
input_time_accu =  0.04315376281738281
lw_time_accu =  0.4367828369140625
accel_time_accu =  0.3981590270996094
output_time_accu =  0.03814697265625
total_time =  0.9162425994873047
input_time_accu =  0.046253204345703125
lw_time_accu =  0.43654441833496094
accel_time_accu =  0.3962516784667969
output_time_accu =  0.03743171691894531
total_time =  0.9164810180664062
input_time_accu =  0.0438690185546875
lw_time_accu =  0.44035911560058594
accel_time_accu =  0.4982948303222656
output_time_accu =  0.03910064697265625
total_time =  1.0216236114501953
input_time_accu =  0.045299530029296875
lw_time_accu =  0.4374980926513672


input_time_accu =  0.04601478576660156
lw_time_accu =  0.4315376281738281
accel_time_accu =  0.3924369812011719
output_time_accu =  0.036716461181640625
total_time =  0.9067058563232422
input_time_accu =  0.04315376281738281
lw_time_accu =  0.45228004455566406
accel_time_accu =  0.3972053527832031
output_time_accu =  0.03695487976074219
total_time =  0.9295940399169922
input_time_accu =  0.045299530029296875
lw_time_accu =  0.5300045013427734
accel_time_accu =  0.4038810729980469
output_time_accu =  0.03814697265625
total_time =  1.0173320770263672
input_time_accu =  0.04267692565917969
lw_time_accu =  0.5347728729248047
accel_time_accu =  0.39649009704589844
output_time_accu =  0.03790855407714844
total_time =  1.0118484497070312
input_time_accu =  0.0438690185546875
lw_time_accu =  0.4324913024902344
accel_time_accu =  0.39577484130859375
output_time_accu =  0.03695487976074219
total_time =  0.9090900421142578
input_time_accu =  0.046253204345703125
lw_time_accu =  0.4327297210693359

input_time_accu =  0.04839897155761719
lw_time_accu =  0.4382133483886719
accel_time_accu =  0.4146099090576172
output_time_accu =  0.03814697265625
total_time =  0.9393692016601562
input_time_accu =  0.044345855712890625
lw_time_accu =  0.43392181396484375
accel_time_accu =  0.39386749267578125
output_time_accu =  0.03790855407714844
total_time =  0.9100437164306641
input_time_accu =  0.045299530029296875
lw_time_accu =  0.4353523254394531
accel_time_accu =  0.4119873046875
output_time_accu =  0.037670135498046875
total_time =  0.9303092956542969
input_time_accu =  0.046253204345703125
lw_time_accu =  0.4525184631347656
accel_time_accu =  0.39386749267578125
output_time_accu =  0.03719329833984375
total_time =  0.9298324584960938
input_time_accu =  0.04458427429199219
lw_time_accu =  0.43272972106933594
accel_time_accu =  0.39124488830566406
output_time_accu =  0.03695487976074219
total_time =  0.9055137634277344
input_time_accu =  0.04649162292480469
lw_time_accu =  0.434160232543945

input_time_accu =  0.04506111145019531
lw_time_accu =  0.4367828369140625
accel_time_accu =  0.40435791015625
output_time_accu =  0.03814697265625
total_time =  0.9243488311767578
input_time_accu =  0.04291534423828125
lw_time_accu =  0.5106925964355469
accel_time_accu =  0.40411949157714844
output_time_accu =  0.03981590270996094
total_time =  0.9975433349609375
input_time_accu =  0.04506111145019531
lw_time_accu =  0.4489421844482422
accel_time_accu =  0.39649009704589844
output_time_accu =  0.037670135498046875
total_time =  0.9281635284423828
input_time_accu =  0.04458427429199219
lw_time_accu =  0.446319580078125
accel_time_accu =  0.396728515625
output_time_accu =  0.03743171691894531
total_time =  0.9250640869140625
input_time_accu =  0.04482269287109375
lw_time_accu =  0.4334449768066406
accel_time_accu =  0.3936290740966797
output_time_accu =  0.037670135498046875
total_time =  0.9095668792724609
input_time_accu =  0.045299530029296875
lw_time_accu =  0.4336833953857422
accel_

input_time_accu =  0.04553794860839844
lw_time_accu =  0.43463706970214844
accel_time_accu =  0.39315223693847656
output_time_accu =  0.03695487976074219
total_time =  0.9102821350097656
input_time_accu =  0.04553794860839844
lw_time_accu =  0.4470348358154297
accel_time_accu =  0.48470497131347656
output_time_accu =  0.038623809814453125
total_time =  1.0159015655517578
input_time_accu =  0.044345855712890625
lw_time_accu =  0.4305839538574219
accel_time_accu =  0.3914833068847656
output_time_accu =  0.03719329833984375
total_time =  0.9036064147949219
input_time_accu =  0.04410743713378906
lw_time_accu =  0.43487548828125
accel_time_accu =  0.3941059112548828
output_time_accu =  0.037670135498046875
total_time =  0.9107589721679688
input_time_accu =  0.04506111145019531
lw_time_accu =  0.4324913024902344
accel_time_accu =  0.3933906555175781
output_time_accu =  0.036716461181640625
total_time =  0.9076595306396484
input_time_accu =  0.04863739013671875
lw_time_accu =  0.4336833953857

input_time_accu =  0.04601478576660156
lw_time_accu =  0.4317760467529297
accel_time_accu =  0.39267539978027344
output_time_accu =  0.03790855407714844
total_time =  0.9083747863769531
input_time_accu =  0.0438690185546875
lw_time_accu =  0.5202293395996094
accel_time_accu =  0.4088878631591797
output_time_accu =  0.03838539123535156
total_time =  1.0113716125488281
input_time_accu =  0.04315376281738281
lw_time_accu =  0.5137920379638672
accel_time_accu =  0.4093647003173828
output_time_accu =  0.04076957702636719
total_time =  1.007080078125
input_time_accu =  0.04315376281738281
lw_time_accu =  0.5216598510742188
accel_time_accu =  0.40793418884277344
output_time_accu =  0.0400543212890625
total_time =  1.0128021240234375
input_time_accu =  0.04267692565917969
lw_time_accu =  0.5211830139160156
accel_time_accu =  0.41103363037109375
output_time_accu =  0.03838539123535156
total_time =  1.0132789611816406
input_time_accu =  0.04315376281738281
lw_time_accu =  1.4297962188720703
acce

input_time_accu =  0.04982948303222656
lw_time_accu =  0.453948974609375
accel_time_accu =  0.3960132598876953
output_time_accu =  0.03838539123535156
total_time =  0.9381771087646484
input_time_accu =  0.04291534423828125
lw_time_accu =  0.4343986511230469
accel_time_accu =  0.39577484130859375
output_time_accu =  0.03743171691894531
total_time =  0.9105205535888672
input_time_accu =  0.0438690185546875
lw_time_accu =  0.4322528839111328
accel_time_accu =  0.3936290740966797
output_time_accu =  0.03695487976074219
total_time =  0.9067058563232422
input_time_accu =  0.04267692565917969
lw_time_accu =  0.4343986511230469
accel_time_accu =  0.39458274841308594
output_time_accu =  0.03743171691894531
total_time =  0.9090900421142578
input_time_accu =  0.04482269287109375
lw_time_accu =  0.4343986511230469
accel_time_accu =  0.3941059112548828
output_time_accu =  0.03719329833984375
total_time =  0.9105205535888672
input_time_accu =  0.04649162292480469
lw_time_accu =  0.43463706970214844


In [ ]:
import pynq
import time

rails = pynq.get_rails()
recorder = pynq.DataRecorder(rails['VCCp0V85'].power)
recorder.frame['VCCp0V85_power'].plot()

In [ ]:
'''
{
    'VCCpSYZYGY_VIO': 
        Rail {name=VCCpSYZYGY_VIO, 
        voltage=Sensor {name=VCCpSYZYGY_VIO_voltage, value=3.376V}, 
        current=Sensor {name=VCCpSYZYGY_VIO_current, value=0.004A}, 
        power=Sensor {name=VCCpSYZYGY_VIO_power, value=0.0W}}, 
    'VDAC_AVCCAUX': 
        Rail {name=VDAC_AVCCAUX, 
        voltage=Sensor {name=VDAC_AVCCAUX_voltage, value=1.824V}, 
        current=Sensor {name=VDAC_AVCCAUX_current, value=0.001A}, 
        power=Sensor {name=VDAC_AVCCAUX_power, value=0.0W}}, 
    'VADC_AVCC': Rail {name=VADC_AVCC, 
        voltage=Sensor {name=VADC_AVCC_voltage, value=0.952V}, 
        current=Sensor {name=VADC_AVCC_current, value=0.012A}, 
        power=Sensor {name=VADC_AVCC_power, value=0.0W}}, 
    'VCCp1V8': Rail {name=VCCp1V8, 
        voltage=Sensor {name=VCCp1V8_voltage, value=1.816V}, 
        current=Sensor {name=VCCp1V8_current, value=0.61A}, 
        power=Sensor {name=VCCp1V8_power, value=1.0W}}, 
    'VCCp0V85': Rail {name=VCCp0V85, 
        voltage=Sensor {name=VCCp0V85_voltage, value=0.868V}, 
        current=Sensor {name=VCCp0V85_current, value=2.48A}, 
        power=Sensor {name=VCCp0V85_power, value=2.0W}}, 
    'VDAC_AVTT': Rail {name=VDAC_AVTT, 
        voltage=Sensor {name=VDAC_AVTT_voltage, value=2.532V}, 
        current=Sensor {name=VDAC_AVTT_current, value=0.01A}, 
        power=Sensor {name=VDAC_AVTT_power, value=0.04W}}, 
    'VADC_AVCCAUX': Rail {name=VADC_AVCCAUX, 
        voltage=Sensor {name=VADC_AVCCAUX_voltage, value=1.824V}, 
        current=Sensor {name=VADC_AVCCAUX_current, value=0.032A}, 
        power=Sensor {name=VADC_AVCCAUX_power, value=0.06W}}, 
    'VDAC_AVCC': Rail {name=VDAC_AVCC, 
        voltage=Sensor {name=VDAC_AVCC_voltage, value=0.944V}, 
        current=Sensor {name=VDAC_AVCC_current, value=0.004A}, 
        power=Sensor {name=VDAC_AVCC_power, value=0.0W}}, 
    'VCCp3V3': Rail {name=VCCp3V3, 
        voltage=Sensor {name=VCCp3V3_voltage, value=3.328V}, 
        current=Sensor {name=VCCp3V3_current, value=0.36A}, 
        power=Sensor {name=VCCp3V3_power, value=1.0W}}
}
'''